## 🧭 Introducere în Markov Decision Processes (MDP)

Un **Markov Decision Process (MDP)** este o formulare matematică a unui mediu în care un agent interacționează în mod secvențial, luând decizii care influențează stările viitoare și recompensele primite.

Un MDP este definit astfel:

$$
\text{MDP} = (S, A, P, R, \gamma)
$$

unde:
- **S** — setul de stări posibile;
- **A** — setul de acțiuni disponibile;
- **P(s'|s, a)** — probabilitatea de a ajunge în starea `s'` după ce agentul alege acțiunea `a` în starea `s`;
- **R(s, a, s')** — recompensa primită pentru tranziția `(s → s')`;
- **γ** — factorul de discount (0 ≤ γ ≤ 1), care controlează importanța viitorului.


---

## 🧠 Proprietatea Markov

MDP-urile respectă **proprietatea Markov**:

> Viitorul depinde doar de prezent, nu și de trecut.

Formal:

$$
P(s_{t+1} | s_t, a_t, s_{t-1}, a_{t-1}, \dots) = P(s_{t+1} | s_t, a_t)
$$

Aceasta presupune că **starea curentă conține toate informațiile relevante** pentru a prezice evoluția sistemului.  
În practică, această condiție este rareori perfect îndeplinită — de aceea, definirea stării devine o cea mai importantă parte în designul mediilor RL.

---

## 🧮 Politici și funcții de valoare

O **politică** π este o strategie de decizie a agentului:

$$
\pi(a|s) = P(A_t = a | S_t = s)
$$

Ea definește distribuția probabilităților asupra acțiunilor posibile în fiecare stare.  
Dacă politica este deterministă, putem scrie simplu:

$$
a = \pi(s)
$$

Funcția de **valoare a unei stări** sub o politică π se definește ca:

$$
V^{\pi}(s) = \mathbb{E}_\pi [ G_t \,|\, S_t = s ]
$$

unde $G_t$ este **returnul** total obținut de agent:

$$
G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \dots
$$

Intuitiv, $V^{\pi}(s)$ ne spune **cât de „bună” este o stare**, dacă agentul continuă să urmeze politica π.

---

## 🔁 Ecuațiile Bellman

Valorile stărilor se leagă între ele prin **ecuațiile Bellman**, care exprimă recurența dintre valoarea curentă și valorile stărilor următoare:

$$
V^{\pi}(s) = \sum_a \pi(a|s) \sum_{s'} P(s'|s,a) \big[ R(s,a,s') + \gamma V^{\pi}(s') \big]
$$

Aceasta este baza tuturor metodelor de evaluare și îmbunătățire a politicilor în RL:
- **Evaluare**: estimăm $V^{\pi}$ pentru o politică dată.  
- **Îmbunătățire**: ajustăm politica pentru a maximiza valoarea stărilor.

---

## 🧭 Obiectivul agentului

Scopul unui agent RL este să găsească **politica optimă**:

$$
\pi^* = \arg\max_\pi V^{\pi}(s)
$$

care maximizează recompensa cumulativă așteptată din orice stare inițială.

În practică, agentul nu cunoaște funcțiile $P$ și $R$, ci le descoperă prin **explorare activă** — acesta fiind motivul pentru care RL se bazează pe învățare din interacțiune, nu din demonstrații explicite.

---

**Pe scurt:**  
- Un MDP este „scheletul” teoretic al Reinforcement Learning-ului.
- MDP definește *cum* acționează agentul, *ce* primește în schimb și *cum* învață să ia decizii mai bune în timp.


## Definirea unui mediu custom (GridWorld)

In [ ]:
import numpy as np
import time

# Definim dimensiunile GridWorld-ului
nrow, ncol = 4, 4
S = nrow * ncol  # numărul total de stări
A = 4  # 0: sus, 1: dreapta, 2: jos, 3: stânga
gamma = 0.99

# Stări terminale: +1 și -1
terminal_plus, terminal_minus = 15, 12

def to_rc(s):
    """Conversie index -> (linie, coloană)"""
    return divmod(s, ncol)

def to_s(r, c):
    """Conversie (linie, coloană) -> index"""
    return r * ncol + c

def step_deterministic(s, a):
    """Tranziție deterministă simplă"""
    if s in (terminal_plus, terminal_minus):
        return s, 0.0, True  # stări terminale
    
    r, c = to_rc(s)
    if a == 0: r = max(0, r - 1)      # sus
    elif a == 1: c = min(ncol - 1, c + 1)  # dreapta
    elif a == 2: r = min(nrow - 1, r + 1)  # jos
    elif a == 3: c = max(0, c - 1)      # stânga
    
    s2 = to_s(r, c)
    if s2 == terminal_plus:
        return s2, +1.0, True
    elif s2 == terminal_minus:
        return s2, -1.0, True
    else:
        return s2, -0.01, False  # mică penalizare pentru fiecare pas

# === Funcție pentru afișarea grilei ===
def render_grid(s, visited=None):
    """Afișează grila 4x4, cu agentul și stările terminale"""
    grid = [["." for _ in range(ncol)] for _ in range(nrow)]
    r, c = to_rc(s)
    grid[r][c] = "A"
    tr, tc = to_rc(terminal_plus)
    grid[tr][tc] = "G"  # goal
    tr, tc = to_rc(terminal_minus)
    grid[tr][tc] = "X"  # pericol
    
    if visited:
        for v in visited:
            if v not in (s, terminal_plus, terminal_minus):
                vr, vc = to_rc(v)
                grid[vr][vc] = "*"
    
    print()
    for row in grid:
        print(" ".join(row))
    print()


## Utilizarea mediului custom cu un agent random

In [ ]:
# === Simulare episod + salvare cadre text ===
rng = np.random.default_rng(42)

def policy(s): return rng.integers(0, A)

def run_episode_with_render(max_steps=50, delay=0.3):
    s = rng.integers(0, S)
    total_reward = 0
    frames = []
    visited = set()
    
    print(f"🎬 Pornim un episod (stare inițială: {s})")
    for t in range(max_steps):
        visited.add(s)
        frame_text = f"\n--- PASUL {t+1} ---\n"
        frame_text += f"Stare: {s}\n"
        frame_text += f"Recompensă totală: {total_reward:.2f}\n"
        frames.append(frame_text)
        print(frame_text)
        render_grid(s, visited)
        time.sleep(delay)
        
        a = policy(s)
        s2, r, done = step_deterministic(s, a)
        total_reward += r
        s = s2
        if done:
            render_grid(s, visited)
            print(f"🏁 Episod terminat după {t+1} pași. Recompensă totală: {total_reward:.2f}")
            break
    return frames

# === Rulează exemplu ===
frames = run_episode_with_render()